In [1]:
import numpy as np
import numpy.testing as npt
from scipy.linalg import expm
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from matplotlib.collections import PatchCollection
import scipy.spatial, scipy.linalg
import scipy.sparse.linalg
from scipy.misc import logsumexp
from scipy.cluster.hierarchy import linkage, dendrogram
import itertools as it
import time
%matplotlib inline


In [2]:
np.sign(np.random.randn(10))

array([-1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.])

In [111]:
class svm_smo(): 
    
    def __init__(self, kernel,C):
        self.kernel = kernel
        self.c = C

    def getkernel(self, X, Y=None):
        n= len(X)
        n2 = n
        if(Y==None):
            X2=np.array(X)
            x2=len(X2)
            n2 = n
        else: 
            X2 = Y
            n2 = len(X2)
 
        if((self.kernel =='gaussian') or (self.kernel ==['gaussian'])):
            w = self.kp
            X1 = (X**2).sum(1).reshape(n,1)*np.ones((n,n2))
            U1 = (X2**2).sum(1).reshape(1,n2) * np.ones([n,n2])
            D = X1 - 2*(X.dot(X2.T)) + U1
            K = np.exp(-D/(2*w**2))
        elif((self.kernel =='polynomial') or (self.kernel ==['polynomial'])):
            p= self.kp
            K = (np.dot(X,X2.T)+1)**p
        elif((self.kernel =='linear') or (self.kernel ==['linear'])):
            K = np.dot(X,X2.T)
        else:
            raise AssertionError("Choose from ['gaussian','polynomial','linear']")
        return K
    
    
    def fx(self,X1,X2,Y):
        K=self.getkernel(X1,X2)
        alpY = (Y*self.alpha).reshape(len(self.alpha),1)
        return np.sign(np.dot(K,alpY)+self.b)
    
    def _compute_box_constraints(self, i, j, Y, alpha, C):
        
        if(Y[i]==Y[j]):
            L = np.max([0,alpha[i]+alpha[j]-C])
            H = np.min([C,alpha[i]+alpha[j]])
        else:
            L = np.max([0,alpha[j]-alpha[i]])
            H = np.min([C,C+alpha[j]-alpha[i]])
        return L, H 
    
    
    def _update_parameters(self, E_i, E_j, i, j, K, Y, alpha, b, C):
        L, H = self._compute_box_constraints( i, j, Y, alpha, C)
        if(L==H):
            return alpha, b, 0
        kappa = 2*K[i,j] -K[i,i] -K[j,j]
        if(kappa>=0):
            return alpha, b, 0
        
        aph2_new= alpha[j]-Y[j]*(E_i-E_j)/kappa
        
        if(aph2_new>H):
            aph2_new=H
        if(aph2_new<L):
            aph2_new=L
            
        aph1_new = alpha[i]+Y[i]*Y[j]*(alpha[j]-aph2_new)
        if(np.abs(alpha[j]-aph2_new)<0.0005):
            return alpha, b, 0
        
        alpha_new = np.array(alpha)
        alpha_new[i]= np.array(aph1_new)
        alpha_new[j]= np.array(aph2_new)
        
        new_b = self._compute_updated_b( E_i, E_j, i, j, K, Y, alpha, alpha_new, b, C)
            
        return alpha_new, new_b, 1
    
    
    def _compute_updated_b(self, E_i, E_j, i, j, K, Y, alpha_old, alpha_new, b_old, C):
        b1 = b_old+E_i+Y[i]*(alpha_new[i]-alpha_old[i])*K[i,i]+Y[j]*(alpha_new[j]-alpha_old[j])*K[i,j]
        b2 = b_old+E_j+Y[i]*(alpha_new[i]-alpha_old[i])*K[i,j]+Y[j]*(alpha_new[j]-alpha_old[j])*K[j,j]
        new_b=(b1+b2)/2
        
        if((alpha_new[i]>0)and(alpha_new[i]<C)):
            new_b = b1
        if((alpha_new[j]>0)and(alpha_new[j]<C)):
            new_b = b2
        
        return new_b
     
        
    def fit(self, X, Y):
        #self.X_fit = X
        N= len(X)
        rang=np.arange(N)
        K = self.getkernel(X)
        self.alpha = np.zeros(N)
        self.b = 1
        P=100
        tol = 0.01
        p=0
        
        while(p<P):
            a=0
            for i in range(N):
                Ei = self.fx(X[i].reshape(1,len(X[i])),X,Y)-Y[i]
                if (((Y[i]*Ei < -tol) and (self.alpha[i] < self.c)) or ( (Y[i]*Ei > tol) and (self.alpha[i] > 0))):
                    j=np.random.choice(np.setdiff1d(rang, np.array([i])),1)
                    Ej = self.fx(X[j].reshape(1,len(X[i])),X,Y)-Y[j]
                    self.alpha, self.b, updated=self._update_parameters( Ei, Ej, i, j, K, Y, self.alpha, self.b, self.c)
                    a = a+updated
            if(a==0):        
                p=p+1
            else:
                p=0
            print(p)
         
        f=self.fx(X,X,Y)
        print(f)
        self.SV= X[np.where(f=1)]
        self.y = Y[np.where(f=1)]
        self.alpha = self.alpha[np.where(f=1)]
        
    def predict(self, X):
        return self.fx(X,self.SV,self.y)

In [112]:

np.random.seed(1)
X_tr = np.hstack((np.random.normal(size=[2, 30]), np.random.normal(size=[2, 30]) + np.array([2., 2.])[:, np.newaxis])).T
Y_tr = np.array([1] * 30 + [-1] * 30)
X_te = np.hstack((np.random.normal(size=[2, 30]), np.random.normal(size=[2, 30]) + np.array([2., 2.])[:, np.newaxis])).T
Y_te = np.array([1] * 30 + [-1] * 30)
C = svm_smo(kernel='linear', C=1.)
C.fit(X=X_tr, Y=Y_tr)
Y_pred = C.predict(X_te)
loss = float(np.sum(np.sign(Y_te) != np.sign(Y_pred)))/float(len(Y_te))
print('test case loss', loss)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
2
0
0
1
0
0
1
0
1
2
3
4
0
1
2
3
0
1
2
0
0
1
2
3
0
1
0
0
0
1
2
3
0
1
2
3
4
0
0
0
1
2
3
4
5
0
1
2
3
4
5
6
7
8
9
0
1
2
0
1
2
3
4
5
6
0
1
2
3
4
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
0
1
2
3
4
5
6
7
8
9
10
11
12
0
1
2
3
4
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39

TypeError: where() takes no keyword arguments

In [ ]:
C.SV.shape